<a href="https://colab.research.google.com/github/sukba-tr/sukba-tr/blob/main/sukru_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/radoslawik/VRPTW_GA_PSO.git

Cloning into 'VRPTW_GA_PSO'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 103 (delta 77), reused 86 (delta 63), pack-reused 0
Receiving objects: 100% (103/103), 57.52 KiB | 14.38 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [ ]:
!pip install deap==1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  ERROR: HTTP error 403 while getting https://files.pythonhosted.org/packages/7a/00/a67c0751b8bda3b419ec942a7cc532bf4bda666b4cfe27eb355e53159190/deap-1.0.0.tar.gz#sha256=724720ababf419a82754b95da765fe228acb382f776e3df74222055d27a28754 (from https://pypi.org/simple/deap/)
ERROR: Could not install requirement deap==1.0 from https://files.pythonhosted.org/packages/7a/00/a67c0751b8bda3b419ec942a7cc532bf4bda666b4cfe27eb355e53159190/deap-1.0.0.tar.gz#sha256=724720ababf419a82754b95da765fe228acb382f776e3df74222055d27a28754 because of HTTP error 403 Client Error: Forbidden for url: https://files.pythonhosted.org/packages/7a/00/a67c0751b8bda3b419ec942a7cc532bf4bda666b4cfe27eb355e53159190/deap-1.0.0.tar.gz for URL https://files.pythonhosted.org/packages/7a/00/a67c0751b8bda3b419ec942a7cc532bf4bda666b4cfe27eb355e53159190/deap-1.0.0.tar.gz#sha256=724720ababf419a82754b95da765fe228acb382f776e3df74222055d

In [ ]:
%cd /content/VRPTW_GA_PSO/

In [ ]:

import os
import io

#BASE_DIR = os.path.abspath(os.path.dirname(__file__))
BASE_DIR = "/content/VRPTW_GA_PSO"
X_COORD = 'x'
Y_COORD = 'y'
COORDINATES = 'coordinates'
INSTANCE_NAME = 'instance_name'
MAX_VEHICLE_NUMBER = 'max_vehicle_number'
VEHICLE_CAPACITY = 'vehicle_capacity'
DEPART = 'depart'
DEPOT = 'depot'
DEMAND = 'demand'
READY_TIME = 'ready_time'
DUE_TIME = 'due_time'
SERVICE_TIME = 'service_time'
DISTANCE_MATRIX = 'distance_matrix'
depot_number = 3
vehicle_count = depot_number

def calculate_distance(cust_1, cust_2):
    x_diff = cust_1[COORDINATES][X_COORD] - cust_2[COORDINATES][X_COORD]
    y_diff = cust_1[COORDINATES][Y_COORD] - cust_2[COORDINATES][Y_COORD]
    return (x_diff**2 + y_diff**2)**0.5


def load_problem_instance(problem_name='E101'):
    cust_num = 0
    depot_count = 0
    text_file = os.path.join(BASE_DIR, 'data', problem_name + '.txt')
    parsed_data = {}
    with io.open(text_file, 'rt', newline='') as fo:
        for line_count, line in enumerate(fo, start=1):
            if line_count == 1:
                parsed_data[INSTANCE_NAME] = line.strip()
            elif line_count == 5:
                values = line.strip().split()
                parsed_data[MAX_VEHICLE_NUMBER] = int(values[0])
                parsed_data[VEHICLE_CAPACITY] = float(values[1])
            elif (10 + depot_number) > line_count > 9:
                # DEPART
                values = line.strip().split()
                parsed_data[F'D_{values[0]}'] = {
                    COORDINATES: {
                        X_COORD: float(values[1]),
                        Y_COORD: float(values[2]),
                    },
                    DEMAND: float(values[3]),
                    READY_TIME: float(values[4]),
                    DUE_TIME: float(values[5]),
                    SERVICE_TIME: float(values[6]),
                }
                parsed_data[F'V_{values[0]}'] = {
                    COORDINATES: {
                        X_COORD: float(values[1]),
                        Y_COORD: float(values[2]),
                    },
                    DEPOT: int(depot_count), # hangi depodan
                }
                depot_count += 1
            elif line_count > (9 + depot_number):
                # CUSTOMERS
                values = line.strip().split()
                parsed_data[F'C_{values[0]}'] = {
                    COORDINATES: {
                        X_COORD: float(values[1]),
                        Y_COORD: float(values[2]),
                    },
                    DEMAND: float(values[3]),
                    READY_TIME: float(values[4]),
                    DUE_TIME: float(values[5]),
                    SERVICE_TIME: float(values[6]),
                }
                cust_num += 1
            else:
                pass

        customers = [F'D_{x}' for x in range(0, depot_count)] + [F'C_{x}' for x in range(1, cust_num+1)]
        parsed_data[DISTANCE_MATRIX] = \
            [[calculate_distance(parsed_data[c1], parsed_data[c2]) for c1 in customers] for c2 in customers]

        return parsed_data, depot_count, vehicle_count


In [ ]:

import random
import operator
import math
import collections

from deap import base, creator, tools
import numpy as np
from random import randrange

def create_route_from_ind(individual, data, depot_count, vehicle_id):

    vehicle_capacity = data[VEHICLE_CAPACITY]
    starting_depot = data[F'V_{vehicle_id}'][DEPOT]
    depart_due_time = data['D_0'][DUE_TIME]

    route = []
    sub_route = []
    route_ending = []
    vehicle_load = 0
    time_elapsed = 0
    previous_cust_id = -depot_count + 1 + starting_depot
    for customer_id in individual:
        demand = data[F'C_{customer_id}'][DEMAND]
        updated_vehicle_load = vehicle_load + demand
        service_time = data[F'C_{customer_id}'][SERVICE_TIME]
        # return_time = min(data[DISTANCE_MATRIX][depot_count+customer_id-1][0:depot_count])
        # depot_id = np.argmin(data[DISTANCE_MATRIX][depot_count+customer_id-1][0:depot_count])
        return_time = data[DISTANCE_MATRIX][depot_count+customer_id-1][starting_depot]
        travel_time = data[DISTANCE_MATRIX][depot_count+previous_cust_id-1][depot_count+customer_id-1]
        provisional_time = time_elapsed + travel_time + service_time + return_time
        # Validate vehicle load and elapsed time
        if (updated_vehicle_load <= vehicle_capacity): # and (provisional_time <= depart_due_time):
            # Add to current sub-route
            sub_route.append(customer_id)
            vehicle_load = updated_vehicle_load
            time_elapsed = provisional_time - return_time
        else:
            # Save current sub-route
            route.append(sub_route)
            # Initialize a new sub-route and add to it
            sub_route = [customer_id]
            vehicle_load = demand
            travel_time = data[DISTANCE_MATRIX][starting_depot][depot_count+customer_id-1]
            route_ending.append(starting_depot)
            time_elapsed = travel_time + service_time
            depart_due_time = data[F'D_{starting_depot}'][DUE_TIME]
        # Update last customer ID
        previous_cust_id = customer_id
    if sub_route:
        # Save current sub-route before return if not empty
        route.append(sub_route)
        route_ending.append(starting_depot)
    return route, route_ending


def create_final_route_from_ind(individual, data, depot_count, vehicle_count):
    final_route = {}
    final_route_ending = {}
    ind_indx = [0, len(individual)-vehicle_count+1] + individual[-vehicle_count+1:]
    ind_indx = sorted(ind_indx)
    for vehicle_id in range(vehicle_count):
      cutted_individual = individual[ind_indx[vehicle_id]:ind_indx[vehicle_id+1]]
      if cutted_individual:
        route, route_ending = create_route_from_ind(cutted_individual, data, depot_count, vehicle_id)
        final_route[vehicle_id] = route
        final_route_ending[vehicle_id] = route_ending
      else:
        final_route[vehicle_id] = []
        final_route_ending[vehicle_id] = []
    return final_route, final_route_ending

def calculate_fitness(individual,data,depot_count, vehicle_count):

    transport_cost = 23.94  # cost of moving 1 vehicle for 1 unit
    vehicle_setup_cost = 500  # cost of adapting new vehicle
    wait_penalty = 0  # penalty for arriving too early
    delay_penalty = 0  # penalty for arriving too late
    total_cost = 0
    fitness = 0
    ind_indx = [0, len(individual)-vehicle_count+1] + individual[-vehicle_count+1:]
    ind_indx = sorted(ind_indx)
    for vehicle_id in range(vehicle_count):
      cutted_individual = individual[ind_indx[vehicle_id]:ind_indx[vehicle_id+1]]
      if cutted_individual:
        route, route_ending = create_route_from_ind(cutted_individual, data, depot_count, vehicle_id)
        total_sub_cost = 999999
        fitness = 0
        max_vehicles_count = data[MAX_VEHICLE_NUMBER]
        # checking if we have enough vehicles
        if len(route) <= max_vehicles_count:
            total_sub_cost = 0
            for route_index, sub_route in enumerate(route):
                sub_route_time_cost = 0
                sub_route_distance = 0
                elapsed_time = 0
                starting_depot = data[F'V_{vehicle_id}'][DEPOT]
                previous_cust_id = -depot_count + 1 + starting_depot
                for cust_id in sub_route:
                    # Calculate section distance
                    distance = data[DISTANCE_MATRIX][depot_count+previous_cust_id-1][depot_count+cust_id-1]
                    # Update sub-route distance
                    sub_route_distance = sub_route_distance + distance

                    # Calculate time cost
                    arrival_time = elapsed_time + distance

                    waiting_time = max(data[F'C_{cust_id}'][READY_TIME] - arrival_time, 0)
                    delay_time = max(arrival_time - data[F'C_{cust_id}'][DUE_TIME], 0)
                    time_cost = wait_penalty * waiting_time + delay_penalty * delay_time

                    # Update sub-route time cost
                    sub_route_time_cost += time_cost

                    # Update elapsed time
                    service_time = data[F'C_{cust_id}'][SERVICE_TIME]
                    elapsed_time = arrival_time + service_time

                    # Update last customer ID
                    previous_cust_id = cust_id

                # Calculate transport cost
                distance_depot = data[DISTANCE_MATRIX][previous_cust_id][route_ending[route_index]]
                sub_route_distance += distance_depot
                sub_route_transport_cost = vehicle_setup_cost + transport_cost * sub_route_distance
                # Obtain sub-route cost
                sub_route_cost = sub_route_time_cost + sub_route_transport_cost
                # Update total cost`
                total_sub_cost += sub_route_cost
        total_cost += total_sub_cost
        # fitness = - math.log(1.0 / total_cost)
    fitness = 100000.0 / total_cost

    return fitness, total_cost


# Double point crossover
def crossover_pmx(ind1, ind2):

    ind_len = len(ind1)
    pos_ind1 = [0]*ind_len
    pos_ind2 = [0]*ind_len

    # position indices list
    for i in range(ind_len):
        pos_ind1[ind1[i]-1] = i
        pos_ind2[ind2[i]-1] = i

    # crossover points
    locus1 = random.randint(0, int(ind_len/2))
    locus2 = random.randint(int(ind_len/2)+1, ind_len-1)

    # crossover
    for i in range(locus1, locus2):
        temp1 = ind1[i]
        temp2 = ind2[i]
        # swap
        ind1[i], ind1[pos_ind1[temp2-1]] = temp2, temp1
        ind2[i], ind2[pos_ind2[temp1-1]] = temp1, temp2
        # save updated positions
        pos_ind1[temp1-1], pos_ind1[temp2-1] = pos_ind1[temp2-1], pos_ind1[temp1-1]
        pos_ind2[temp1-1], pos_ind2[temp2-1] = pos_ind2[temp2-1], pos_ind2[temp1-1]

    return ind1, ind2


# Chose 2 indexes and swap values between them
def mutate_swap(individual):

    ind_len = len(individual)
    locus1 = random.randint(0, int(ind_len / 2))
    locus2 = random.randint(int(ind_len / 2) + 1, ind_len - 1)

    temp = individual[locus1]
    individual[locus1] = individual[locus2]
    individual[locus2] = temp

    return individual,


# The initialization consist in generating a random position and a random speed for a particle.
# The next function creates a particle and initializes its attributes,
# except for the attribute best, which will be set only after evaluation
def generate_particle(size, val_min, val_max, s_min, s_max, vehicle_count):
    vals = list(range(val_min, val_max + 1))
    random.shuffle(vals)
    for i in range(vehicle_count-1):
      vals += [randrange(size)]
    part = creator.Particle(vals)
    part.speed = [random.uniform(s_min, s_max) for _ in range(len(vals))]
    part.smin = s_min
    part.smax = s_max
    return part


def create_particle(vals, s_min, s_max):
    part = creator.Particle(vals)
    part.speed = [random.uniform(s_min, s_max) for _ in range(len(vals))]
    part.smin = s_min
    part.smax = s_max
    return part


def remove_duplicates(vals):
    duplic = [item for item, count in collections.Counter(vals).items() if count > 1]
    uniq_part = []
    offset = 0.001
    count = [1] * len(duplic)
    for val in vals:
        if val in duplic:
            ind = duplic.index(val)
            val += offset * count[ind]
            count[ind] += 1
        uniq_part.append(val)

    return uniq_part



#bunu da değiştirdim
# Change floats to integers and deal with duplicates
def validate_particle(particle,vehicle_count):

    unique_part = remove_duplicates(particle[:-vehicle_count+1])
    indx = particle[-vehicle_count+1:]
    sorted_asc = sorted(unique_part, key=float)
    validated_part = []
    if len(sorted_asc) > len(set(sorted_asc)):
        print("problem")

    for val in unique_part:
        index = sorted_asc.index(val)
        validated_part.append((index + 1))
    for id in indx:
      new_id = np.minimum(len(unique_part)-1,np.maximum(0,round(id)))

      validated_part.append((new_id))
    return validated_part


def validate_particle2(particle):
    swap_list = remove_duplicates(list(map(operator.add, list(range(1, len(particle)+1)), particle.speed)))
    validated_part = []
    validated_speed = []
    sorted_asc = sorted(swap_list, key=float)

    if len(sorted_asc) > len(set(sorted_asc)):
        print("problem")
    for val in swap_list:
        index = sorted_asc.index(val)
        validated_part.append(particle[index])
        validated_speed.append(particle.speed[index])

    return validated_part, validated_speed

# The function updateParticle() first computes the speed,
# then limits the speed values between smin and smax,
# and finally computes the new particle position.

# random indexler için burayı değiştirdik. speed hesaplayıp ekliyot
def update_particle(part, best, phi1, phi2, vehicle_count):

    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    #print("part ", part, " best ", part)
    # the particle's best position
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    # the neighbourhood best
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    # update particle speed
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    # speed limits
    for i, speed in enumerate(part.speed):
        if abs(speed) < part.smin:
            part.speed[i] = math.copysign(part.smin, speed)
            # adjust maximum speed if necessary
        elif abs(speed) > part.smax:
            part.speed[i] = math.copysign(part.smax, speed)
    #print("part2 ", part)
    #print("part_s_2 ", part.speed)
    new_part = list(map(operator.add, part, part.speed))
    part[:] = validate_particle(new_part,vehicle_count)
    #print("n_part ", new_part)
    #print( " v_part ", part)

    # part[:], part.speed[:] = validate_particle2(part)






ModuleNotFoundError: ignored

In [ ]:
from deap import base, creator, tools
import matplotlib.pyplot as plt
import numpy
import time


# plot data from given problem with chosen customer amount
def plot_instance(instance_name, customer_number):

    instance, depot_count, vehicle_count = load_problem_instance(instance_name)

    for depot_id in range(0,depot_count):
        coordinates = [instance[F'D_{depot_id}'][COORDINATES][X_COORD],
                       instance[F'D_{depot_id}'][COORDINATES][Y_COORD]]
        dep, = plt.plot(coordinates[0], coordinates[1], 'kP', label='depot')

    for customer_id in range(1,customer_number):
        coordinates = [instance[F'C_{customer_id}'][COORDINATES][X_COORD],
                       instance[F'C_{customer_id}'][COORDINATES][Y_COORD]]
        custs, = plt.plot(coordinates[0], coordinates[1], 'ro', label='customers')
    plt.ylabel("y coordinate")
    plt.xlabel("x coordinate")
    plt.legend([dep, custs], ['depot', 'customers'], loc=1)
    plt.show()


# plot the result
def plot_route(route, instance_name,route_ending,depot_count,vehicle_id):

    instance, depot_count, vehicle_count = load_problem_instance(instance_name)
    color_pack = ['bo', 'go', 'ro', 'co', 'mo', 'yo', 'ko',
                  'b*', 'g*', 'r*', 'c*', 'm*', 'y*', 'k*',
                  'bp', 'gp', 'rp', 'cp', 'mp', 'yp', 'kp']
    line_color_pack = ['b', 'g', 'r', 'c', 'm', 'y', 'k',
                       'b', 'g', 'r', 'c', 'm', 'y', 'k',
                       'b', 'g', 'r', 'c', 'm', 'y', 'k']
    c_ind = -1

    for depot_id in range(0,depot_count):
        coordinates = [instance[F'D_{depot_id}'][COORDINATES][X_COORD],
                       instance[F'D_{depot_id}'][COORDINATES][Y_COORD]]

        dep, = plt.plot(coordinates[0], coordinates[1], 'kP', label='depot')
    # coordinates = [instance[F'D_{vehicle_id}'][COORDINATES][X_COORD],
    #                    instance[F'D_{vehicle_id}'][COORDINATES][Y_COORD]]

    # dep, = plt.plot(coordinates[0], coordinates[1], 'kP', label='depot')
    for route_idx, single_route in enumerate(route):
        c_ind += 1
        if route_idx == 0:
          depot_id = vehicle_id
          coordinates = [instance[F'D_{depot_id}'][COORDINATES][X_COORD],
                       instance[F'D_{depot_id}'][COORDINATES][Y_COORD]]
        else:
          depot_id = route_ending[route_idx-1]
          coordinates = [instance[F'D_{depot_id}'][COORDINATES][X_COORD],
                       instance[F'D_{depot_id}'][COORDINATES][Y_COORD]]
        depot_id_2 = route_ending[route_idx]
        depot = [instance[F'D_{depot_id_2}'][COORDINATES][X_COORD],
                  instance[F'D_{depot_id_2}'][COORDINATES][Y_COORD]]
        for customer_id in single_route:
            prev_coords = coordinates
            coordinates = [instance[F'C_{customer_id}'][COORDINATES][X_COORD],
                           instance[F'C_{customer_id}'][COORDINATES][Y_COORD]]
            plt.plot(coordinates[0], coordinates[1], color_pack[c_ind])
            plt.arrow(prev_coords[0], prev_coords[1], coordinates[0] - prev_coords[0],
                      coordinates[1] - prev_coords[1], color=line_color_pack[c_ind],
                      length_includes_head=True, head_width=1, head_length=2)
        plt.arrow(coordinates[0], coordinates[1], depot[0] - coordinates[0],
                  depot[1] - coordinates[1], color=line_color_pack[c_ind],
                  length_includes_head=True, head_width=1, head_length=2)
    plt.ylabel("y coordinate")
    plt.xlabel("x coordinate")
    plt.show()
    return

def plot_final_route(route, instance_name,route_ending,depot_count,vehicle_count):
  for vehicle_id in range(vehicle_count):
    print("Depot ID: " + str(vehicle_id))
    plot_route(route[vehicle_id], instance_name,route_ending[vehicle_id],depot_count,vehicle_id)

# printing the solution
def print_route(route, route_ending,vehicle_id):
    route_num = 0
    for route_idx , sub_route in enumerate(route):
        route_num += 1
        if route_idx == 0:
          single_route = f'D.{str(vehicle_id)}'
        else:
          single_route = f'D.{route_ending[route_idx-1]}'
        for customer_id in sub_route:
            single_route = f'{single_route} -> C.{customer_id}'
        single_route = f'{single_route} -> D.{route_ending[route_idx]}'
        print(f' # Vehicle {route_num} # {single_route}')

def print_final_route(route, route_ending,vehicle_count):
  for vehicle_id in range(vehicle_count):
    print("Depot ID: " + str(vehicle_id))
    print_route(route[vehicle_id],route_ending[vehicle_id],vehicle_id)
# runs the pso and prints the solution
# https://deap.readthedocs.io/en/master/examples/pso_basic.html
# Once the operators are registered in the toolbox, we can fire up the algorithm by firstly creating a new population,
# and then apply the original PSO algorithm.
# The variable best contains the best particle ever found (it is known as gbest in the original algorithm).
def run_pso(instance_name, particle_size, pop_size, max_iteration,
            cognitive_coef, social_coef, s_limit=3, plot=False, save=False, logs=False):

    instance, depot_count, vehicle_count = load_problem_instance(instance_name)

    if instance is None:
        return

    if plot:
        plot_instance(instance_name=instance_name, customer_number=particle_size)

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Particle", list, fitness=creator.FitnessMax, speed=list,
                   smin=None, smax=None, best=None)

    toolbox = base.Toolbox()
    toolbox.register("particle", generate_particle,
                     size=particle_size, val_min=1, val_max=particle_size, s_min=-s_limit, s_max=s_limit, vehicle_count = vehicle_count)
    toolbox.register("population", tools.initRepeat, list, toolbox.particle)
    toolbox.register("update", update_particle, phi1=cognitive_coef, phi2=social_coef,vehicle_count = vehicle_count)
    toolbox.register('evaluate', calculate_fitness, data=instance, depot_count = depot_count, vehicle_count = vehicle_count)

    pop = toolbox.population(n=pop_size)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    best = None
    iter_num = 0
    previous_best = 0

    print('### EVOLUTION START ###')
    start = time.time()

    for g in range(max_iteration):

        fit_count = 0
        for part in pop:
            #print("created ", len(part))
            part.fitness.values = toolbox.evaluate(part)
            if part.fitness.values[0] > previous_best:
                previous_best = part.fitness.values[0]
                iter_num = g + 1
            elif part.fitness.values[0] == previous_best:
                fit_count += 1

        if fit_count > int(numpy.ceil(pop_size * 0.15)):
            rand_pop = toolbox.population(n=pop_size)
            for part in rand_pop:
                part.fitness.values = toolbox.evaluate(part)
            some_inds = tools.selRandom(rand_pop, int(numpy.ceil(pop_size * 0.1)))  # random pop here
            mod_pop = tools.selWorst(pop, int(numpy.ceil(pop_size * 0.9)))

        else:
            some_inds = tools.selBest(pop, int(numpy.ceil(pop_size * 0.05)))  # elite pop here
            mod_pop = tools.selRandom(pop, int(numpy.ceil(pop_size * 0.95)))
        mod_pop = list(map(toolbox.clone, mod_pop))

        for part in mod_pop:
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values

            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values

        for part in mod_pop:
            #print("part_b  ", part)
            toolbox.update(part, best)
            #print("part_a  ", part)

        mod_pop.extend(some_inds)
        pop[:] = mod_pop
        #print("++++++++++++")
        if (not (g%100)) and g>50:
          print("iter: ", g)
          end = time.time()
          print('### EVOLUTION END ###')
          best_ind = tools.selBest(pop, 1)[0]
          print(f'Best individual: {best_ind}')
          final_route, final_route_ending = create_final_route_from_ind(best_ind, instance, depot_count,vehicle_count)
          print_final_route(final_route, final_route_ending,vehicle_count)

          print(f'Fitness: { round(best_ind.fitness.values[0],2) }')
          print(f'Total cost: { round(calculate_fitness(best_ind, instance,depot_count,vehicle_count)[1],2) }')
          print(f'Found in (iteration): { iter_num }')
          print(f'Execution time (s): { round(end - start,2) }')

          #if plot:
            #plot_final_route(route=final_route, instance_name=instance_name, route_ending = final_route_ending, depot_count = depot_count, vehicle_count = vehicle_count)

        # Gather all the stats in one list and print them
        logbook.record(gen=g+1, evals=len(pop), **stats.compile(pop))
        #print(logbook.stream)

    end = time.time()
    print('### EVOLUTION END ###')
    best_ind = tools.selBest(pop, 1)[0]
    print(f'Best individual: {best_ind}')
    final_route, final_route_ending = create_final_route_from_ind(best_ind, instance, depot_count,vehicle_count)
    print_final_route(final_route, final_route_ending,vehicle_count)

    print(f'Fitness: { round(best_ind.fitness.values[0],2) }')
    print(f'Total cost: { round(calculate_fitness(best_ind, instance,depot_count,vehicle_count)[1],2) }')
    print(f'Found in (iteration): { iter_num }')
    print(f'Execution time (s): { round(end - start,2) }')
    # print(f'{round(best_ind.fitness.values[0], 2)} & {round(calculate_fitness(best_ind, instance)[1],2)} & {iter_num} & {round(end - start, 2)}')

    #if plot:
    #    plot_final_route(route=final_route, instance_name=instance_name, route_ending = final_route_ending, depot_count = depot_count, vehicle_count = vehicle_count)

    return final_route


In [ ]:
import sys

sys.argv = ["hadi","C151","PSO"]
if __name__ == '__main__':

    if len(sys.argv) != 3:
        print("invalid arguments")
        sys.exit()

    random.seed(231)
    plot_result = True

    problem_name = str(sys.argv[1])
    alg_name = str(sys.argv[2])



    customers_count = 92
    max_generations = 1500

    particles_pop_size = 100
    cognitive_acceleration = 2
    social_acceleration = 2
    speed_limit = 1.5

    population_size = 150
    crossover_prob = 0.9
    mutation_prob = 0.09

    print('### GENERAL INFO ###')
    print('Problem name: ' + problem_name)
    print(f'Customer count: {customers_count}')
    print(f'Max iterations: {max_generations}')
    print('Algorithm: ' + alg_name)
    print('### ALGORITHM PARAMETERS ###')

    '''
    for mutation_prob in [0.03, 0.09]:
    for crossover_prob in [0.5, 0.7, 0.9]:
    for speed_limit in [1.5, 3]:
    for cognitive_acceleration, social_acceleration in [[2, 2], [1.5, 2.5], [2.5, 1.5]]:
    '''

    if alg_name == "PSO":
        print(f'Particles population size: {particles_pop_size}')
        print(f'Social acceleration: {social_acceleration}')
        print(f'Cognitive acceleration: {cognitive_acceleration}')
        print(f'Speed limit: {speed_limit}')
        res = run_pso(instance_name=problem_name, particle_size=customers_count, pop_size=particles_pop_size,
                      max_iteration=max_generations, cognitive_coef=cognitive_acceleration,
                      social_coef=social_acceleration, s_limit=speed_limit, plot=plot_result)
    else:
        print("invalid algorithm")
        sys.exit()

In [ ]:
parsed_data, depot_count, vehicle_count = load_problem_instance(problem_name='C150')

In [ ]:
parsed_data[DISTANCE_MATRIX]